# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-03-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-02-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-11-03 06:22:09,33.93911,67.709953,156307,7281,NaN,NaN,Afghanistan,401.524946,4.658141
1,NaN,NaN,NaN,Albania,2021-11-03 06:22:09,41.15330,20.168300,186222,2937,NaN,NaN,Albania,6470.984780,1.577150
2,NaN,NaN,NaN,Algeria,2021-11-03 06:22:09,28.03390,1.659600,206649,5927,NaN,NaN,Algeria,471.252189,2.868148
3,NaN,NaN,NaN,Andorra,2021-11-03 06:22:09,42.50630,1.521800,15516,130,NaN,NaN,Andorra,20081.537566,0.837845
4,NaN,NaN,NaN,Angola,2021-11-03 06:22:09,-11.20270,17.873900,64487,1713,NaN,NaN,Angola,196.210291,2.656349


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,155944,156040,156071,156124,156166,156196,156210,156250,156284,156307
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,181696,181960,182610,183282,183873,184340,184887,185300,185497,186222
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,205822,205903,205990,206069,206160,206270,206358,206452,206566,206649


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7255,7260,7262,7266,7268,7269,7272,7280,7281,7281
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2874,2880,2888,2893,2902,2909,2916,2924,2931,2937
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,5890,5894,5899,5904,5907,5913,5918,5920,5924,5927


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10047,10048,10061,10083,10219,10228,10239,10252,10258,10271
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,37258,37266,37285,37325,37350,37372,37398,37414,37425,37445
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,3578,3578,3583,3595,3596,3596,3599,3601,3601,3605


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,148,148,148,148,148,148,148,148,148,148
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,552,552,552,552,552,555,555,555,555,558
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,76,76,76,76,76,76,76,76,76,76


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
673,1001.0,Autauga,Alabama,US,2021-11-03 06:22:09,32.539527,-86.644082,10271,148,NaN,NaN,"Autauga, Alabama, US",18384.077037,1.440950
710,1075.0,Lamar,Alabama,US,2021-11-03 06:22:09,33.779950,-88.096680,2347,49,NaN,NaN,"Lamar, Alabama, US",17001.086563,2.087772
711,1077.0,Lauderdale,Alabama,US,2021-11-03 06:22:09,34.901719,-87.656247,14671,297,NaN,NaN,"Lauderdale, Alabama, US",15821.371955,2.024402


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,46169012,748518,0.0
India,34308140,459191,0.0
Brazil,21821124,608071,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,46169012,748518,0.0,45420494.0,2021-11-03 06:22:09,37.934269,-91.444083
India,34308140,459191,0.0,33848949.0,2021-11-03 06:22:09,23.088275,81.806127
Brazil,21821124,608071,0.0,21213053.0,2021-11-03 06:22:09,-12.669522,-48.480493
United Kingdom,9174153,141390,0.0,9032763.0,2021-11-03 06:22:09,28.164647,-33.966498
Russia,8455232,236462,0.0,8218770.0,2021-11-03 06:22:09,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,4936466,72188,0.0
Texas,4247209,71380,0.0
Florida,3690812,59670,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,4936466,72188,0.0,4864278.0,2021-11-03 06:22:09,37.843962,-120.728594
Texas,4247209,71380,0.0,4175829.0,2021-11-03 06:22:09,31.660643,-98.653069
Florida,3690812,59670,0.0,3631142.0,2021-11-03 06:22:09,28.940755,-82.700744
New York,2567747,56533,0.0,2511214.0,2021-11-03 06:22:09,42.544151,-75.474183
Illinois,1704031,28291,0.0,1675740.0,2021-11-03 06:22:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1495014,26661,0.0
Arizona,Maricopa,741940,12230,0.0
Florida,Miami-Dade,678911,6472,0.0
Illinois,Cook,644255,11303,0.0
Texas,Harris,576561,9164,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1495014,26661,0.0,1468353.0,2021-11-03 06:22:09,34.308284,-118.228241,6037.0
Arizona,Maricopa,741940,12230,0.0,729710.0,2021-11-03 06:22:09,33.348359,-112.491815,4013.0
Florida,Miami-Dade,678911,6472,0.0,672439.0,2021-11-03 06:22:09,25.611236,-80.551706,12086.0
Illinois,Cook,644255,11303,0.0,632952.0,2021-11-03 06:22:09,41.841448,-87.816588,17031.0
Texas,Harris,576561,9164,0.0,567397.0,2021-11-03 06:22:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,156250,185300,206452,15516,64433,4058,5288807,308326,172030,830981,...,9100442,393899,185963,6,407151,921122,452997,9791,209734,132977
2021-11-01,156284,185497,206566,15516,64458,4062,5289945,309397,173165,835504,...,9140441,394053,186244,6,407866,926720,453782,9799,209739,133007
2021-11-02,156307,186222,206649,15516,64487,4069,5291285,310629,174276,840902,...,9174153,394248,186793,6,408727,932357,454172,9806,209760,133047


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,7280,2924,5920,130,1710,102,115950,6328,1743,11357,...,141055,6078,1323,1,4891,22083,4663,1889,3661,4678
2021-11-01,7281,2931,5924,130,1713,102,115989,6379,1756,11369,...,141098,6080,1325,1,4902,22131,4681,1890,3661,4679
2021-11-02,7281,2937,5927,130,1713,102,116010,6441,1768,11380,...,141390,6080,1329,1,4915,22205,4689,1894,3661,4683


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-11-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/24/21,10/25/21,10/26/21,10/27/21,10/28/21,10/29/21,10/30/21,10/31/21,11/1/21,11/2/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10047,10048,10061,10083,10219,10228,10239,10252,10258,10271
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,2244,2248,2257,2296,2310,2321,2323,2324,2325,2347
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,14314,14323,14366,14582,14627,14636,14641,14650,14653,14671
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,4324,4327,4352,4533,4558,4562,4571,4572,4574,4596
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,23536,23540,23556,25172,25232,25237,25254,25283,25284,25308


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,832047,137887,4,1168900,513215,4915796,740461,402583,144198,49,...,1280265,4238438,549882,40191,7204,924771,725435,272532,882457,102926
2021-11-01,832264,139429,4,1170990,513352,4932199,747855,403415,144397,49,...,1280265,4242156,553061,40340,7206,927999,730108,273030,887242,103622
2021-11-02,833009,139925,4,1172105,513993,4936466,750251,403827,144473,49,...,1283557,4247209,554311,40513,7233,929244,731894,273659,890607,103926


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-10-31       10252   37414    3601  4279  10406    1526   3351   22348   
2021-11-01       10258   37425    3601  4281  10408    1526   3353   22350   
2021-11-02       10271   37445    3605  4283  10423    1526   3365   22341   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-10-31         5785     3068  ...    5100   1374     5409     1365   
2021-11-01         5786     3069  ...    5142   1391     5444     1371   
2021-11-02         5787     3071  ...    5142   1399     5478     1371   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-10-31           7575  5204  3829          0     1731   1109  
2021-11-01           7617  5218  3843          0     1745   1121  
2021-11-02           7617  5221  3853          0     1756   1126  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,15573,725,0,21154,8376,71950,8186,8764,2091,0,...,16342,71193,3208,364,81,13907,8628,4426,9416,1174
2021-11-01,15573,728,0,21155,8384,72111,8258,8766,2091,0,...,16342,71262,3237,368,81,13984,8659,4450,9430,1174
2021-11-02,15629,734,0,21156,8412,72188,8292,8776,2095,0,...,16400,71380,3248,369,81,14020,8693,4475,9467,1207


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-10-31         148     555      76   87    178      44     96     495   
2021-11-01         148     555      76   87    178      44     96     495   
2021-11-02         148     558      76   89    179      44     96     497   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-10-31          143       61  ...      70     25       49       16   
2021-11-01          143       61  ...      70     25       49       16   
2021-11-02          144       61  ...      70     25       50       16   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-10-31             80    14    26          0       35      9  
2021-11-01             80    14    26          0       35      9  
2021-11-02             80    14    27          0       35      9  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,0.000256,0.002234,0.000456,0.0,0.000917,0.000000,0.000104,0.005174,0.008595,0.006887,...,0.004163,0.000460,0.001702,0.0,0.002245,0.006028,0.000000,0.001227,0.000057,0.000384
2021-11-01,0.000218,0.001063,0.000552,0.0,0.000388,0.000986,0.000215,0.003474,0.006598,0.005443,...,0.004395,0.000391,0.001511,0.0,0.001756,0.006077,0.001733,0.000817,0.000024,0.000226
2021-11-02,0.000147,0.003908,0.000402,0.0,0.000450,0.001723,0.000253,0.003982,0.006416,0.006461,...,0.003688,0.000495,0.002948,0.0,0.002111,0.006083,0.000859,0.000714,0.000100,0.000301


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,0.001100,0.002743,0.000338,0.0,0.001171,0.0,0.000069,0.007002,0.004611,0.001058,...,0.000525,0.000165,0.001514,0.0,0.001433,0.002406,0.000000,0.004787,0.0,0.000642
2021-11-01,0.000137,0.002394,0.000676,0.0,0.001754,0.0,0.000336,0.008059,0.007458,0.001057,...,0.000305,0.000329,0.001512,0.0,0.002249,0.002174,0.003860,0.000529,0.0,0.000214
2021-11-02,0.000000,0.002047,0.000506,0.0,0.000000,0.0,0.000181,0.009719,0.006834,0.000968,...,0.002069,0.000000,0.003019,0.0,0.002652,0.003344,0.001709,0.002116,0.0,0.000855


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,0.000474,0.000000,0.0,0.002436,0.000431,0.000216,0.000000,0.000000,0.001723,0.0,...,0.000000,0.000094,0.000000,0.006713,0.000000,0.000000,0.000000,0.002431,0.000000,0.000000
2021-11-01,0.000261,0.011183,0.0,0.001788,0.000267,0.003337,0.009986,0.002067,0.001380,0.0,...,0.000000,0.000877,0.005781,0.003707,0.000278,0.003491,0.006442,0.001827,0.005422,0.006762
2021-11-02,0.000895,0.003557,0.0,0.000952,0.001249,0.000865,0.003204,0.001021,0.000526,0.0,...,0.002571,0.001191,0.002260,0.004289,0.003747,0.001342,0.002446,0.002304,0.003793,0.002934


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-10-31      0.001270  0.000428  0.000556  0.000234  0.001058     0.0   
2021-11-01      0.000585  0.000294  0.000000  0.000467  0.000192     0.0   
2021-11-02      0.001267  0.000534  0.001111  0.000467  0.001441     0.0   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-10-31      0.000597  0.000313  0.000865  0.000000  ...  0.000000   
2021-11-01      0.000597  0.000089  0.000173  0.000326  ...  0.008235   
2021-11-02      0.003579 -0.000403  0.000173  0.000652  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-10-31      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-11-01      0.012373  0.006471  0.004396   0.005545  0.002690  0.003656   
2021-11-02      0.005751  0.006245  0.000000   0.000000  0.000575  0.002602   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-10-31            NaN  0.000000  0.000000  
2021-11-01            NaN  0.008088  0.010821  
2021-11-02            NaN  0.006304  0.004460  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,0.000000,0.000000,NaN,0.000047,0.000717,0.000125,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000759,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2021-11-01,0.000000,0.004138,NaN,0.000047,0.000955,0.002238,0.008796,0.000228,0.000000,NaN,...,0.000000,0.000969,0.009040,0.010989,0.0,0.005537,0.003593,0.005423,0.001487,0.000000
2021-11-02,0.003596,0.008242,NaN,0.000047,0.003340,0.001068,0.004117,0.001141,0.001913,NaN,...,0.003549,0.001656,0.003398,0.002717,0.0,0.002574,0.003927,0.005618,0.003924,0.028109


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2021-10-31         0.0  0.000000     0.0  0.000000  0.000000     0.0    0.0   
2021-11-01         0.0  0.000000     0.0  0.000000  0.000000     0.0    0.0   
2021-11-02         0.0  0.005405     0.0  0.022989  0.005618     0.0    0.0   

Province_State                              ... Wyoming                   \
Admin2          Calhoun  Chambers Cherokee  ...    Park Platte  Sheridan   
2021-10-31      0.00000  0.000000      0.0  ...     0.0    0.0  0.000000   
2021-11-01      0.00000  0.000000      0.0  ...     0.0    0.0  0.000000   
2021-11-02      0.00404  0.006993      0.0  ...     0.0    0.0  0.020408   

Province_State                                                                 
Admin2         Sublette Sweetwater Teton     Uinta Unassigned Washakie Weston  
2021-10-31          0.0        0.0   0.0  0.000000        NaN      0.0    0.0  
2021-11-01          0.0        0.0   0.0  0.000000        NaN      0.0    0.0  
2021-11-02          0.0        0.0   0.0  0.038462        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,0.000210,0.002583,0.000453,0.000350,0.001077,0.001260,0.000157,0.006122,0.008691,0.006829,...,0.004399,0.000579,0.001819,0.015625,0.002868,0.005747,0.000212,0.001305,0.000077,0.000298
2021-11-01,0.000214,0.001823,0.000503,0.000175,0.000733,0.001123,0.000186,0.004798,0.007645,0.006136,...,0.004397,0.000485,0.001665,0.007812,0.002312,0.005912,0.000972,0.001061,0.000051,0.000262
2021-11-02,0.000181,0.002866,0.000452,0.000087,0.000591,0.001423,0.000220,0.004390,0.007030,0.006298,...,0.004043,0.000490,0.002306,0.003906,0.002211,0.005998,0.000916,0.000888,0.000075,0.000281


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,0.000716,0.002599,0.000589,6.566955e-24,0.001012,0.000993,0.000095,0.007290,0.006402,0.001197,...,0.000882,0.000119,0.001839,0.0,0.002548,0.002574,0.000361,0.003486,0.000073,0.000343
2021-11-01,0.000427,0.002496,0.000632,3.283478e-24,0.001383,0.000496,0.000216,0.007675,0.006930,0.001127,...,0.000593,0.000224,0.001675,0.0,0.002399,0.002374,0.002111,0.002008,0.000037,0.000278
2021-11-02,0.000213,0.002272,0.000569,1.641739e-24,0.000692,0.000248,0.000198,0.008697,0.006882,0.001047,...,0.001331,0.000112,0.002347,0.0,0.002525,0.002859,0.001910,0.002062,0.000018,0.000567


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-10-31,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2021-11-01,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2021-11-02,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,0.000911,0.001437,0.002604,0.002571,0.000696,0.000549,0.001165,0.000257,0.002018,5.250549e-180,...,0.000270,0.000496,0.000850,0.005693,0.000572,0.000439,0.000914,0.002882,0.000742,0.001224
2021-11-01,0.000586,0.006310,0.001302,0.002180,0.000481,0.001943,0.005575,0.001162,0.001699,2.625274e-180,...,0.000135,0.000686,0.003316,0.004700,0.000425,0.001965,0.003678,0.002355,0.003082,0.003993
2021-11-02,0.000741,0.004934,0.000651,0.001566,0.000865,0.001404,0.004389,0.001091,0.001113,1.312637e-180,...,0.001353,0.000939,0.002788,0.004494,0.002086,0.001653,0.003062,0.002329,0.003437,0.003463


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-10-31      0.001966  0.000551  0.000635  0.000516  0.001735  0.000055   
2021-11-01      0.001276  0.000422  0.000318  0.000492  0.000964  0.000027   
2021-11-02      0.001272  0.000478  0.000714  0.000480  0.001202  0.000014   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-10-31      0.001322  0.000530  0.001601  0.000751  ...  0.001090   
2021-11-01      0.000959  0.000310  0.000887  0.000539  ...  0.004663   
2021-11-02      0.002269 -0.000046  0.000530  0.000595  ...  0.002331   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-10-31      0.000469  0.001673  0.000966   0.001103  0.000779  0.000598   
2021-11-01      0.006421  0.004072  0.002681   0.003324  0.001734  0.002127   
2021-11-02      0.006086  0.005159  0.001340   0.001662  0.001155  0.002365   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-10-31           -1.0  0.001467  0.003642  
2021-11-01           -1.0  0.004778  0.007231  
2021-11-02           -1.0  0.005541  0.005846  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-10-31,0.001301,0.001543,NaN,0.001248,0.001266,0.000392,0.000792,0.000204,0.001390,NaN,...,0.000510,0.001245,0.001208,0.002141,0.000591,0.000710,0.001104,0.001759,0.000681,0.000352
2021-11-01,0.000650,0.002841,NaN,0.000647,0.001110,0.001315,0.004794,0.000216,0.000695,NaN,...,0.000255,0.001107,0.005124,0.006565,0.000296,0.003123,0.002348,0.003591,0.001084,0.000176
2021-11-02,0.002123,0.005541,NaN,0.000347,0.002225,0.001191,0.004456,0.000678,0.001304,NaN,...,0.001902,0.001382,0.004261,0.004641,0.000148,0.002849,0.003137,0.004604,0.002504,0.014142


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2021-10-31      0.000013  0.000696  1.836478e-06  0.001453  0.001068   
2021-11-01      0.000007  0.000348  9.182391e-07  0.000727  0.000534   
2021-11-02      0.000003  0.002877  4.591195e-07  0.011858  0.003076   

Province_State                                                    ...  \
Admin2           Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2021-10-31      0.000743 -0.001921  0.002009  0.001761  0.002094  ...   
2021-11-01      0.000372 -0.000961  0.001004  0.000880  0.001047  ...   
2021-11-02      0.000186 -0.000480  0.002522  0.003937  0.000523  ...   

Province_State   Wyoming                                                     \
Admin2              Park    Platte  Sheridan  Sublette Sweetwater     Teton   
2021-10-31      0.000031  0.000020  0.000341  0.001060   0.000209  0.000009   
2021-11-01      0.000015  0.000010  0.000171  0.000530   0.000104  0.000005   
2021-11-02      0.000008  0.000005  0.010289  0.000265   0.000052  0.000002   

Province_State                                               
Admin2                 Uinta Unassigned  Washakie    Weston  
2021-10-31      3.031292e-10   0.752941  0.000975  0.000015  
2021-11-01      1.515646e-10   0.752941  0.000487  0.000008  
2021-11-02      1.923077e-02   0.752941  0.000244  0.000004  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 